We use the *Malý lexikon obcí České republiky (2023)* provided by the Český statistický úřad (https://csu.gov.cz/) as our data source for the Czech municipalities names.

The data is available as an Excel spreadsheet at https://csu.gov.cz/docs/107508/2a9acd50-9f4d-bcc7-49a9-a3389e5f9242/32019924012cz.xlsx

In [1]:
import pandas as pd
import yaml, os, sys
import itertools

notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = notebook_dir

if project_root not in sys.path:
    sys.path.append(project_root)

df = pd.read_excel(project_root + '/data/raw/32019924012cz.xlsx', header=1)
df = df[['Kód kraje', 'Správní obvod obce\ns rozšířenou působností\n(SO ORP)', 'Název obce']].dropna()
df.rename(columns={'Kód kraje': 'region_code', 
                   'Správní obvod obce\ns rozšířenou působností\n(SO ORP)': 'district',
                   'Název obce': 'municipality'}, inplace=True)

df.replace('P R A H A', 'Praha', inplace=True) # Je draha. It's spelled this way in the original spreadsheet.

# Some cities names, such as Brno, are spelled in all caps (BRNO) in the spreadsheet.
# We reformat such instances to the usual spelling.
# Note that prepositions (předložky) are not supposed to be capitalized.
format_municipality_name = lambda name: ' '.join(word.title() if word.isupper() else word for word in name.strip().split()) 

df['municipality'] = df['municipality'].apply(format_municipality_name)
df.head(10)

,region_code,district,municipality
5,CZ010,x,Praha
7,CZ020,Benešov,Benešov
8,CZ020,Benešov,Bukovany
9,CZ020,Benešov,Bystřice
10,CZ020,Benešov,Čakov
11,CZ020,Benešov,Čerčany
12,CZ020,Benešov,Český Šternberk
13,CZ020,Benešov,Čtyřkoly
14,CZ020,Benešov,Divišov
15,CZ020,Benešov,Drahňovice


In [2]:
# There are localities that share the same name while being in different districts.
# For instance, the lexikon lists 14 villages named Nová Ves, 9 villages named Němčice and 9 named Petrovice.
# On a related note, there is also 'Nová Ves u Nového Města na Moravě',
# which happens to be the municipality with the longest name in the Czech Republic at present.

df_grouped = df.groupby('municipality')['district'].apply(list).reset_index()

very_common_names = df_grouped['district'].apply(len).nlargest(10).index
print (df_grouped.loc[very_common_names])
df_grouped.loc[very_common_names]['district'].apply(len)

     municipality                                           district
2744     Nová Ves  [Brandýs n. Lab.-St.Boleslav, Kralupy nad Vlta...
2823      Němčice  [Kolín, Mladá Boleslav, Prachatice, Strakonice...
3066    Petrovice  [Rakovník, Sedlčany, Ústí nad Labem, Nový Bydž...
3732      Slatina  [Kladno, Horažďovice, Kralovice, Lovosice, Mor...
413       Březina  [Mnichovo Hradiště, Jindřichův Hradec, Rokycan...
3033       Pavlov  [Kladno, Jihlava, Pelhřimov, Světlá nad Sázavo...
3640       Sedlec  [Brandýs n. Lab.-St.Boleslav, Mladá Boleslav, ...
4598       Vrbice  [Poděbrady, Vimperk, Karlovy Vary, Roudnice na...
16         Babice  [Říčany, Prachatice, Nový Bydžov, Moravské Bud...
422       Březová  [Hořovice, Karlovy Vary, Sokolov, Uherský Brod...


2744    14
2823     9
3066     9
3732     8
413      7
3033     7
3640     7
4598     7
16       6
422      6
Name: district, dtype: int64

In [3]:
# Municipalities with curiously long names.
df_grouped[df_grouped['municipality'].str.split().apply(len) > 3]

,municipality,district
6,Albrechtice v Jizerských horách,[Tanvald]
29,Bartošovice v Orlických horách,[Rychnov nad Kněžnou]
94,Blatnice pod Svatým Antonínkem,[Veselí nad Moravou]
219,Brandýs nad Labem -Stará Boleslav,[Brandýs n. Lab.-St.Boleslav]
366,Bílý Kostel nad Nisou,[Liberec]
634,Deštné v Orlických horách,[Dobruška]
675,Dobrá Voda u Hořic,[Hořice]
676,Dobrá Voda u Pacova,[Pacov]
677,Dobrá Voda u Českých Budějovic,[České Budějovice]
879,Dvůr Králové nad Labem,[Dvůr Králové nad Labem]


We append the names of Prague districts (*městské části*). The data is taken from the *Statistický lexikon obcí (2024)*. 

Source: https://csu.gov.cz/docs/107508/ac5474bb-dd94-83c1-5376-4a6b8a193cd8/17200124_data.zip

TODO: Add Prague streets and public places.

In [4]:
# 17200124k5_01

def remove_par_suffix(s):
    i = s.find('(')
    if i >= 0:
        s = s[:i]
    return ' '.join(s.split())

df_praha = pd.read_excel(project_root + '/data/raw/17200124k5_01.xlsx', header=0)
praha_casti = df_praha.iloc[:, 2].dropna().apply(remove_par_suffix).unique() 

df_praha_casti = pd.DataFrame({'municipality': praha_casti})
df_praha_casti['district'] = [['Praha']] * len(df_praha_casti)
# Merging two parts.
df_grouped = pd.concat([df_grouped, df_praha_casti], ignore_index=True)

# Removing possible duplicates.
df_grouped = df_grouped.groupby('municipality', as_index=False)['district'].agg(district=lambda arr: list(itertools.chain.from_iterable(arr)))

df_grouped.to_csv(project_root + '/data/clean/geo_base.csv', index=False)
df_grouped.head(10), df_grouped.tail(10)


(                      municipality                             district
 0                         Abertamy                             [Ostrov]
 1                           Adamov  [Čáslav, České Budějovice, Blansko]
 2                         Adršpach                            [Broumov]
 3                         Albertov                              [Praha]
 4                      Albrechtice                 [Lanškroun, Havířov]
 5           Albrechtice nad Orlicí                [Kostelec nad Orlicí]
 6          Albrechtice nad Vltavou                              [Písek]
 7  Albrechtice v Jizerských horách                            [Tanvald]
 8                     Albrechtičky                            [Bílovec]
 9                          Aloisov                              [Praha],
           municipality                                           district
 6325           Žítková                                     [Uherský Brod]
 6326             Žíšov                       